## The data came from census Bureau of United States

In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_rows',None)
import json

In [55]:
import csv
dfdata = pd.read_csv("new_work_in_out_county.csv")

In [56]:
dfdata.head()

,GEO_ID,NAME,B08130_001E,B08130_001M,B08130_002E,B08130_002M,B08130_003E,B08130_003M,B08130_004E,B08130_004M,...,B08130_027E,B08130_027M,B08130_028E,B08130_028M,B08130_029E,B08130_029M,B08130_030E,B08130_030M,B08130_031E,B08130_031M
0,id,Geographic Area Name,Estimate!!Total,Margin of Error!!Total,Estimate!!Total!!Worked in state of residence,Margin of Error!!Total!!Worked in state of res...,Estimate!!Total!!Worked in state of residence!...,Margin of Error!!Total!!Worked in state of res...,Estimate!!Total!!Worked in state of residence!...,Margin of Error!!Total!!Worked in state of res...,...,"Estimate!!Total!!Taxicab, motorcycle, bicycle,...","Margin of Error!!Total!!Taxicab, motorcycle, b...","Estimate!!Total!!Taxicab, motorcycle, bicycle,...","Margin of Error!!Total!!Taxicab, motorcycle, b...","Estimate!!Total!!Taxicab, motorcycle, bicycle,...","Margin of Error!!Total!!Taxicab, motorcycle, b...","Estimate!!Total!!Taxicab, motorcycle, bicycle,...","Margin of Error!!Total!!Taxicab, motorcycle, b...",Estimate!!Total!!Worked at home,Margin of Error!!Total!!Worked at home
1,0500000US01001,"Autauga County, Alabama",24325,553,24204,569,8842,682,15362,767,...,310,191,114,83,196,163,15,21,601,179
2,0500000US01003,"Baldwin County, Alabama",87837,1690,82091,1629,64615,1557,17476,1109,...,671,297,611,290,60,70,329,146,4961,688
3,0500000US01005,"Barbour County, Alabama",8683,450,8321,450,6520,440,1801,232,...,130,66,106,65,24,22,20,31,115,50
4,0500000US01007,"Bibb County, Alabama",8024,477,7969,479,3147,452,4822,489,...,136,82,95,77,41,36,0,21,118,71


In [57]:
dfdata = dfdata.drop(0)

In [60]:
dfdata2 = dfdata.loc[:, ['GEO_ID','NAME','B08130_003E','B08130_004E']]
dfdata2.head()

,GEO_ID,NAME,B08130_003E,B08130_004E
1,0500000US01001,"Autauga County, Alabama",8842,15362
2,0500000US01003,"Baldwin County, Alabama",64615,17476
3,0500000US01005,"Barbour County, Alabama",6520,1801
4,0500000US01007,"Bibb County, Alabama",3147,4822
5,0500000US01009,"Blount County, Alabama",6981,14124


In [61]:
dfdata2.shape

(3220, 4)

In [62]:
dfdata2.dropna(inplace=True)

In [63]:
dfdata3 = dfdata2.rename(columns = {'B08130_003E':'Worked_inside_county','B08130_004E':'Worked_outside_county'})
dfdata3.head()

,GEO_ID,NAME,Worked_inside_county,Worked_outside_county
1,0500000US01001,"Autauga County, Alabama",8842,15362
2,0500000US01003,"Baldwin County, Alabama",64615,17476
3,0500000US01005,"Barbour County, Alabama",6520,1801
4,0500000US01007,"Bibb County, Alabama",3147,4822
5,0500000US01009,"Blount County, Alabama",6981,14124


In [108]:
dfdata3['Worked_inside_county'] = pd.to_numeric(dfdata3['Worked_inside_county'])
dfdata3['Worked_outside_county'] = pd.to_numeric(dfdata3['Worked_outside_county'])

In [207]:
dfdata3['Attrition_rate'] = dfdata3.apply(lambda x: x['Worked_outside_county']/x['Worked_inside_county'], axis=1)
dfdata3.head()

,GEO_ID,NAME,Worked_inside_county,Worked_outside_county,Attrition_rate
1,0500000US01001,"Autauga County, Alabama",8842,15362,1.737390
2,0500000US01003,"Baldwin County, Alabama",64615,17476,0.270464
3,0500000US01005,"Barbour County, Alabama",6520,1801,0.276227
4,0500000US01007,"Bibb County, Alabama",3147,4822,1.532253
5,0500000US01009,"Blount County, Alabama",6981,14124,2.023206


In [208]:
dfdata3.shape

(3220, 5)

In [209]:
import folium
import os

In [211]:
# Create choropleth map for attrition rate
map_1 = folium.Map(location=[39, -96], zoom_start=4)
high_res_county_geo = os.path.relpath('gz_2010_us_050_00_500k.json') # from http://eric.clst.org/Stuff/USGeoJSON
# Add choropleth layer
map_1.choropleth(
 geo_data=high_res_county_geo,
 name='choropleth',
 data=dfdata3,
 columns=['GEO_ID', 'Attrition_rate'],
 key_on='properties.GEO_ID',
 fill_color='YlOrRd',
#  fill_color='Set2',
 fill_opacity=0.8,
 line_opacity=0.05,
 legend_name='Attrition_rate',
#  threshold_scale=[0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,1,1.3,1.5,1.7,2,2.1,2.2,2.3,2.5,3,5,7,10]
#  threshold_scale=[0,0.5, 1, 1.5, 2, 2.5, 3.5, 4, 4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10]
#  threshold_scale=[0,0.2,0.5,0.7,1,1.5,2,3.5,5,10]
#  threshold_scale=[0,20,50,70,150,200,250,500,700,1000]
#    threshold_scale=[0,100,200,300,400,500,600,700,850]
    threshold_scale=[0,0.2,0.5,0.7,1.5,2,3,5,7,9]
)
folium.LayerControl().add_to(map_1)

map_1.save('choropleth_map2.html')

In [221]:
# Create choropleth map for work outside county
map_2 = folium.Map(location=[39, -96], zoom_start=4)
high_res_county_geo = os.path.relpath('gz_2010_us_050_00_500k.json') # from http://eric.clst.org/Stuff/USGeoJSON
# Add choropleth layer
map_2.choropleth(
 geo_data=high_res_county_geo,
 name='choropleth',
 data=dfdata3,
 columns=['GEO_ID', 'Worked_outside_county'],
 key_on='properties.GEO_ID',
 fill_color='YlGnBu',
#  fill_color='Set2',
 fill_opacity=0.8,
 line_opacity=0.05,
 legend_name='Worked_outside_county',
#  threshold_scale=[0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,1,1.3,1.5,1.7,2,2.1,2.2,2.3,2.5,3,5,7,10]
#  threshold_scale=[0,0.5, 1, 1.5, 2, 2.5, 3.5, 4, 4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10]
#  threshold_scale=[0,0.2,0.5,0.7,1,1.5,2,3.5,5,10]
#  threshold_scale=[0,20,50,70,150,200,250,500,700,1000]
#    threshold_scale=[0,100,200,300,400,500,600,700,850]
    threshold_scale=[0,15000,25000,35000,55000,118000,218000,418000,618000]
)
folium.LayerControl().add_to(map_2)

map_2.save('choropleth_map_out.html')

In [237]:
# Create choropleth map for work inside county
map_3 = folium.Map(location=[39, -96], zoom_start=4)
high_res_county_geo = os.path.relpath('gz_2010_us_050_00_500k.json') # from http://eric.clst.org/Stuff/USGeoJSON
# Add choropleth layer
map_3.choropleth(
 geo_data=high_res_county_geo,
 name='choropleth',
 data=dfdata3,
 columns=['GEO_ID', 'Worked_inside_county'],
 key_on='properties.GEO_ID',
 fill_color='YlOrRd',
#  fill_color='Set2',
 fill_opacity=0.8,
 line_opacity=0.05,
 legend_name='Worked_inside_county',
#  threshold_scale=[0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,1,1.3,1.5,1.7,2,2.1,2.2,2.3,2.5,3,5,7,10]
#  threshold_scale=[0,0.5, 1, 1.5, 2, 2.5, 3.5, 4, 4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10]
#  threshold_scale=[0,0.2,0.5,0.7,1,1.5,2,3.5,5,10]
#  threshold_scale=[0,20,50,70,150,200,250,500,700,1000]
#    threshold_scale=[0,100,200,300,400,500,600,700,850]
   threshold_scale=[0,150000,250000,350000,550000,700000,1500000,2500000,4500000]
)
folium.LayerControl().add_to(map_3)

map_3.save('choropleth_map_in2.html')

In [117]:
dfdata.head()

,GEO_ID,NAME,B08130_001E,B08130_001M,B08130_002E,B08130_002M,B08130_003E,B08130_003M,B08130_004E,B08130_004M,...,B08130_027E,B08130_027M,B08130_028E,B08130_028M,B08130_029E,B08130_029M,B08130_030E,B08130_030M,B08130_031E,B08130_031M
1,0500000US01001,"Autauga County, Alabama",24325,553,24204,569,8842,682,15362,767,...,310,191,114,83,196,163,15,21,601,179
2,0500000US01003,"Baldwin County, Alabama",87837,1690,82091,1629,64615,1557,17476,1109,...,671,297,611,290,60,70,329,146,4961,688
3,0500000US01005,"Barbour County, Alabama",8683,450,8321,450,6520,440,1801,232,...,130,66,106,65,24,22,20,31,115,50
4,0500000US01007,"Bibb County, Alabama",8024,477,7969,479,3147,452,4822,489,...,136,82,95,77,41,36,0,21,118,71
5,0500000US01009,"Blount County, Alabama",21230,713,21105,713,6981,557,14124,674,...,62,36,25,24,37,28,17,19,441,153


In [129]:
dfdata4 = dfdata.loc[:, ['B08130_008E','B08130_013E','B08130_018E','B08130_023E','B08130_028E','B08130_009E','B08130_014E','B08130_019E','B08130_024E','B08130_029E']]
dfdata4.head()

,B08130_008E,B08130_013E,B08130_018E,B08130_023E,B08130_028E,B08130_009E,B08130_014E,B08130_019E,B08130_024E,B08130_029E
1,7082,905,14,126,114,13739,1416,0,11,196
2,53498,4859,58,628,611,16424,962,13,17,60
3,5338,785,0,176,106,1575,176,22,4,24
4,2717,172,24,21,95,4162,594,25,0,41
5,5747,670,5,93,25,12602,1461,24,0,37


In [130]:
dfdata5 = dfdata4.rename(columns = {'B08130_008E':'Drovealone_incounty','B08130_013E':'Carpool_incounty','B08130_018E':'Public_transportation_incounty','B08130_023E':'Walked_incounty','B08130_028E':'Other_incounty',
                                    'B08130_009E':'Drovealone_outcounty','B08130_014E':'Carpool_outcounty','B08130_019E':'Public_transportation_outcounty','B08130_024E':'Walked_outcounty','B08130_029E':'Other_outcounty'})
dfdata5.head()

,Drovealone_incounty,Carpool_incounty,Public_transportation_incounty,Walked_incounty,Other_incounty,Drovealone_outcounty,Carpool_outcounty,Public_transportation_outcounty,Walked_outcounty,Other_outcounty
1,7082,905,14,126,114,13739,1416,0,11,196
2,53498,4859,58,628,611,16424,962,13,17,60
3,5338,785,0,176,106,1575,176,22,4,24
4,2717,172,24,21,95,4162,594,25,0,41
5,5747,670,5,93,25,12602,1461,24,0,37


In [131]:
dfdata5.shape

(3220, 10)

In [138]:
dfdata5.columns

Index(['Drovealone_incounty', 'Carpool_incounty',
       'Public_transportation_incounty', 'Walked_incounty', 'Other_incounty',
       'Drovealone_outcounty', 'Carpool_outcounty',
       'Public_transportation_outcounty', 'Walked_outcounty',
       'Other_outcounty'],
      dtype='object')

In [143]:
for col in dfdata5.columns:
    dfdata5[col] = pd.to_numeric(dfdata5[col])

In [149]:
dfdata5['test'] = 1

,Drovealone_incounty,Carpool_incounty,Public_transportation_incounty,Walked_incounty,Other_incounty,Drovealone_outcounty,Carpool_outcounty,Public_transportation_outcounty,Walked_outcounty,Other_outcounty,test
1,7082,905,14,126,114,13739,1416,0,11,196,1
2,53498,4859,58,628,611,16424,962,13,17,60,1
3,5338,785,0,176,106,1575,176,22,4,24,1
4,2717,172,24,21,95,4162,594,25,0,41,1
5,5747,670,5,93,25,12602,1461,24,0,37,1
6,1477,438,0,256,0,1647,226,11,9,58,1
7,4844,596,0,63,54,1436,264,0,4,13,1
8,30731,2704,87,549,300,7862,1233,0,27,82,1
9,4889,505,11,68,32,3404,591,0,20,36,1
10,4125,407,27,29,0,2013,404,0,0,0,1


In [152]:
dfdata6 = dfdata5.groupby('test').sum()
dfdata6

,Drovealone_incounty,Carpool_incounty,Public_transportation_incounty,Walked_incounty,Other_incounty,Drovealone_outcounty,Carpool_outcounty,Public_transportation_outcounty,Walked_outcounty,Other_outcounty
test,,,,,,,,,,
1,80827217,9710726,4368248,3839787,2100794,29500800,3434769,2532221,169907,379293


In [156]:
dfdata6.reset_index()

,test,Drovealone_incounty,Carpool_incounty,Public_transportation_incounty,Walked_incounty,Other_incounty,Drovealone_outcounty,Carpool_outcounty,Public_transportation_outcounty,Walked_outcounty,Other_outcounty
0,1,80827217,9710726,4368248,3839787,2100794,29500800,3434769,2532221,169907,379293


In [158]:
dfdata6.to_csv("transportation.csv")

In [176]:
from pyecharts import options as opts
from pyecharts.charts import Radar

In [213]:
v1=[[9710726,4368248,3839787,2100794]]   
v2=[[3434769,2532221,169907,379293]]
radar1=(
    Radar()
    .add_schema(# add schema
        schema=[
            opts.RadarIndicatorItem(name='Carpool',color = '#313695'),
            opts.RadarIndicatorItem(name='Public_transportation',color = '#313695'),
            opts.RadarIndicatorItem(name='Walk',color = '#313695'),
            opts.RadarIndicatorItem(name='Taxicab, motorcycle, bicycle, or other',color = '#313695')
        ]
    )
    .add('worked_inside_county',v1,color = '#f9713c')
    .add('worked_outside_county',v2,color = '#b3e4a1') 
    .set_global_opts(title_opts=opts.TitleOpts(title='Different ways to commute'))
)

radar1.render('radar_chart.html')


'/Users/yuxuanyuan/Documents/PITT FILE/Information Visualization/finalproject/radar_chart.html'

## Use javascript to adjust the figure's size, lengend position

In [217]:
# "radar": {
#         "indicator": [
#             {
#                 "name": "Carpool",
#                 "color": "#313695"
#             },
#             {
#                 "name": "Public_transportation",
#                 "color": "#313695"
#             },
#             {
#                 "name": "Walk",
#                 "color": "#313695"
#             },
#             {
#                 "name": "Taxicab, motorcycle, bicycle, or other",
#                 "color": "#313695"
#             }
#         ],
#         "name": {
#             "textStyle": {}
#         },
#         "splitLine": {
#             "show": true,
#             "lineStyle": {
#                 "width": 1,
#                 "opacity": 1,
#                 "curveness": 0,
#                 "type": "solid"
#             }
#         },
#         "splitArea": {
#             "show": true,
#             "areaStyle": {
#                 "opacity": 0
#             }
#         },
#         "axisLine": {
#             "show": true,
#             "onZero": true,
#             "onZeroAxisIndex": 0
#         }
#     }